In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import (
    model_selection,
    preprocessing,
    dummy,
    metrics,
    ensemble,
    tree,
    neighbors,
    pipeline,
    compose,
    linear_model,
    svm,
)
import xgboost as xgb

In [ ]:
CAT_COLS = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependants",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
]
NUM_COLS = ["tenure", "MonthlyCharges", "TotalCharges"]
TARGET = ["Churn"]
SEED = 69420

df = pd.read_csv(
    "../data/WA_Fn-UseC_-Telco-Customer-Churn.csv", usecols=CAT_COLS + NUM_COLS + TARGET
)

In [ ]:
CAT_COLS_OHE = ["PaymentMethod", "Contract", "InternetService"]
CAT_COLS_LE = CAT_COLS - CAT_COLS_OHE

In [ ]:
def to_object(x):
    return np.array(x).astype("float32")


col_preprocessor = compose.ColumnTransformer(
    transformers=[
        (
            "change_dtype_TotalCharges_",
            preprocessing.FunctionTransformer(to_object),
            ["TotalCharges"],
        ),
        ("num_col_StandardScaler_", preprocessing.StandardScaler(), NUM_COLS),
        ("cat_col_OneHotEncoder_", preprocessing.OneHotEncoder(), CAT_COLS_OHE),
        ("cat_col_LabelEncoder_", preprocessing.LabelEncoder(), CAT_COLS),
    ]
)

In [ ]:
X, y = df.drop(columns=TARGET), df[TARGET]

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=SEED
)

In [ ]:
def run_experiments(
    models, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, seed=SEED
) -> pd.DataFrame:
    results = dict()
    for name, model in models:
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, predictions)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            y_test, predictions, average="weighted"
        )
        print(f"{name} --> {fscore}")
        results[name] = (accuracy, precision, recall, fscore)
    return pd.DataFrame.from_dict(results)


models = [
    (
        "dummy_classifier",
        dummy.DummyClassifier(random_state=SEED, strategy="most_frequent"),
    ),
    ("k_nearest_neighbors", neighbors.KNeighborsClassifier()),
    (
        "logistic_regression",
        linear_model.LogisticRegression(
            random_state=SEED, solver="liblinear", class_weight="balanced"
        ),
    ),
    ("support_vector_machines", svm.SVC(random_state=SEED, kernel="rbf")),
    ("random_forest", ensemble.RandomForestClassifier(random_state=SEED)),
    ("gradient_boosting", ensemble.GradientBoostingClassifier(random_state=SEED)),
    ("decision_tree", tree.DecisionTreeClassifier(random_state=SEED)),
    ("adaboost", ensemble.AdaBoostClassifier()),
    (
        "voting",
        ensemble.VotingClassifier(
            estimators=[
                ("gbc", ensemble.GradientBoostingClassifier()),
                ("lr", linear_model.LogisticRegression()),
                ("abc", ensemble.AdaBoostClassifier()),
            ],
            voting="soft",
        ),
    ),
]

results = run_experiments(models)
results.columns = ["accuracy", "precision", "recall", "fscore"]
results = results.sort_values(by=["fscore"], ascending=False)
results